In [12]:
# Setup
if __name__ == '__main__':
    import os
    # Change the current working directory to the parent directory of this file
    os.chdir(os.path.dirname(os.path.dirname(os.path.dirname(__vsc_ipynb_file__))))

from evaluation import get_actual_demand
from system_state import SystemState
from utils import load_problem_data
import numpy as np
import pandas as pd
import uuid
import json

demand, datacenters, servers, selling_prices = load_problem_data()
system_state = SystemState(datacenters, servers)

seed = 123
np.random.seed(seed)
actual_demand = get_actual_demand(demand)

In [11]:
# Simulate the algorithm with the most profitable server/latency
from greedy_profit_v2.data import get_sorted_servers, break_even_time_all
from greedy_profit_v2.results import save_results_as_actions


results = []
sorted_servers = get_sorted_servers('data/test_data/most_profitable_servers_by_artem.csv')

for server_generation, latency_sensitivity in sorted_servers:
    remaining_demand = actual_demand.copy()
    if 'CPU' in server_generation:
        if server_generation in ['CPU.S1']:
            continue
        slots_size = 2
    else:
        slots_size = 4

    if latency_sensitivity == 'low':
        datacenter_id = 'DC1'
    elif latency_sensitivity == 'medium':
        datacenter_id = 'DC2'
    elif latency_sensitivity == 'high':
        datacenter_id = 'DC3'

    print(f"Server generation: {server_generation}, Latency sensitivity: {latency_sensitivity}")
    while True:
        # 1) Find the ranges of time steps between which this server/latency is in demand
        relevant_demand = remaining_demand.query(f'server_generation == @server_generation and {latency_sensitivity} > 0')
        # print(relevant_demand)
        time_steps_of_demand = relevant_demand.get('time_step').to_numpy()
        # print(time_steps_of_demand)

        # # DEBUG
        # # Rmove a single time step from the middle of the range (to prove this step works for ranges with gaps of no demand)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 20)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 20)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 20)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 20)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 20)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 30)
        # time_steps_of_demand = np.delete(time_steps_of_demand, 58)

        time_steps_diff = np.diff(time_steps_of_demand)
        gap_indices = np.append(np.where(time_steps_diff > 1), len(time_steps_of_demand) - 1)

        ranges = []
        start = 0
        for gap in gap_indices:
            ranges.append((time_steps_of_demand[start], time_steps_of_demand[gap]))
            start = gap + 1

        # print(ranges)

        # 2) Merge ranges which have a negligibly small gap in between (relative to the length of the smallest range)
        i = 0
        while i < len(ranges) - 1:
            length = ranges[i][1] - ranges[i][0]
            length_next = ranges[i + 1][1] - ranges[i + 1][0]
            if ranges[i + 1][0] - ranges[i][1] < (min(length, length_next)/4 * 17):
                ranges[i] = (ranges[i][0], ranges[i + 1][1])
                ranges.pop(i + 1)
            else:
                i += 1
        # print(ranges)

        # 3) Filter all ranges which last for less than the time it takes for the server/latency to break even
        break_even_time = break_even_time_all[server_generation][latency_sensitivity]
        ranges = [r for r in ranges if r[1] - r[0] >= (2*break_even_time)]

        # print(ranges)

        
        # 3) For each range (from longest to shortest):
        sorted_ranges_i = np.argsort([r[1] - r[0] for r in ranges])
        for i in reversed(sorted_ranges_i):
            # i = 2
            current_range = ranges[i]
            # print(range)

            # 1) Calculate the minimum demand across that range
            demand_in_range = relevant_demand.query(f'time_step >= @current_range[0] and time_step <= @current_range[1]')
            min_demand = demand_in_range.min()[latency_sensitivity]
            # 2) Calculate the number of servers to buy meet the minimum demand
            capacity = servers.set_index('server_generation').loc[server_generation]['capacity']
            desired_buy_count = int(np.round(min_demand / capacity))

            # print(f"{min_demand}/{capacity} = {min_demand / capacity} ~~ {str(desired_buy_count)} GPUs to buy")


            # 4) Store the number of servers to buy, which data centre, the buy time step, the dismiss time step
            results.append({
                'server_generation': server_generation,
                'buy_count': str(desired_buy_count),
                'datacenter_id': datacenter_id,
                'buy_time_step': str(current_range[0]),
                'dismiss_time_step': str(current_range[1] + 1)
            })


            # 5) For each demand in the range, subtract the capacity * number of servers to buy
            demand_to_subtract = desired_buy_count * capacity
            # print(f"Subtracting {demand_to_subtract} from the demand in the range")
            for index, row in demand_in_range.iterrows():
                remaining = row[latency_sensitivity] - demand_to_subtract
                remaining_demand.at[index, latency_sensitivity] = remaining


            # 6) Filter new demand values which are too low to buy at least 1 server for
            remaining_demand = remaining_demand.query(f'{latency_sensitivity} > {(capacity / 2) + 1}')


            # break
        # 4) Repeat steps 1.1 to 1.3.4 with the new demand values until there are no ranges after 1.2
        if len(ranges) == 0:
            # print("No more ranges of demand to satisfy")
            results_df = pd.DataFrame(results)
            total_servers_bought = results_df['buy_count'].astype(int).sum()
            print(f"Total servers bought: {total_servers_bought}")
            break

# save_json('./results.json', results)
save_results_as_actions('./gpuall_cpus432_min_length_x2_merge_threshold_x17_seed_123.json', results)

Server generation: GPU.S3, Latency sensitivity: high
Total servers bought: 484
Server generation: GPU.S3, Latency sensitivity: medium
Total servers bought: 758
Server generation: GPU.S3, Latency sensitivity: low
Total servers bought: 791
Server generation: GPU.S2, Latency sensitivity: high
Total servers bought: 849
Server generation: GPU.S1, Latency sensitivity: high
Total servers bought: 1643
Server generation: GPU.S2, Latency sensitivity: medium
Total servers bought: 1770
Server generation: GPU.S1, Latency sensitivity: medium
Total servers bought: 1904
Server generation: GPU.S2, Latency sensitivity: low
Total servers bought: 2001
Server generation: GPU.S1, Latency sensitivity: low
Total servers bought: 2171
Server generation: CPU.S4, Latency sensitivity: high
Total servers bought: 3433
Server generation: CPU.S3, Latency sensitivity: high
Total servers bought: 4081
Server generation: CPU.S4, Latency sensitivity: medium
Total servers bought: 6826
Server generation: CPU.S3, Latency sens